In [1]:
# !pip install requests
# !pip install opencv-python
# !pip install mediapipe
# !pip install pygame
# !pip install setuptools wheel
# !pip install playsound

In [2]:
#import

import cv2
import mediapipe as mp
import time
import requests
from requests.auth import HTTPBasicAuth
from urllib.parse import quote
from requests.auth import HTTPBasicAuth
import math
from playsound import playsound
import os
import pygame
import numpy as np
import io

from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from pathlib import Path


pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\enric\Documents\GitHub\CV\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [3]:
# pygame player

pygame.init()
pygame.mixer.init()

#variabili globali 

is_paused=False
playlist = []
current_song_index = 0
volume = 0.5




def cerca_canzoni(lista_titoli):
    global playlist
    playlist = []
    
    for titolo in lista_titoli:
        url = f"https://api.deezer.com/search?q={titolo}"
        risposta = requests.get(url)
        dati = risposta.json()
        
        if dati['data']:
            brano = dati['data'][0]
            info = {
                "titolo": brano['title'],
                "artista": brano['artist']['name'],
                "preview_url": brano['preview']
            }
            playlist.append(info)
            print(f"🎶 Aggiunto: {info['titolo']} - {info['artista']}")
        else:
            print(f"⚠️ Nessun risultato per: {titolo}")

def load_song(index):
    global volume
    if 0 <= index < len(playlist):
        url = playlist[index]['preview_url']
        risposta = requests.get(url)
        if risposta.status_code == 200:
            audio = io.BytesIO(risposta.content)
            pygame.mixer.music.load(audio)
            pygame.mixer.music.set_volume(volume)
            pygame.mixer.music.play()
            print(f"▶️ Riproduzione: {playlist[index]['titolo']} - {playlist[index]['artista']}")
        else:
            print("❌ Errore nel caricamento della preview.")
    else:
        print("❌ Indice fuori range.")

        
def play_pause():
    global is_paused

    if is_paused:
            pygame.mixer.music.unpause()
            is_paused = False
            print("▶️ Ripresa")
    else:
            pygame.mixer.music.pause()
            is_paused = True
            print("⏸️ Pausa")
    



def next_song():
    global current_song_index, is_paused
    current_song_index = (current_song_index + 1) % len(playlist)
    load_song(current_song_index)
    is_paused = False
    

def previous_song():
    global current_song_index, is_paused
    current_song_index = (current_song_index - 1) % len(playlist)
    load_song(current_song_index)
    is_paused = False

def volume_up():
    global volume
    volume = round(min(1.0, (volume + 0.1)),1)
    print(volume)
    pygame.mixer.music.set_volume(volume)
    # print(f"🔊 Volume: {int(volume * 100)}%")

def volume_down():
    global volume
    volume = round(max(0.0, (volume - 0.1)),1)
    pygame.mixer.music.set_volume(volume)
    print("volume",volume)
    # print(f"🔉 Volume: {int(volume * 100)}%")



In [4]:
# avvia playlist
#Per creare una playlist bisogna inserire i titoli e il cantante\gruppo della canzone nella lista sottostante, avviando il codice si avrá la
# possibilitá di riprodurre le canzoni per 30 secondi (preview versione demo)
titoli = ["Bohemian Rhapsody Queen",
          "Around the World Daft Punk",
            "Blinding Lights The Weeknd",
            "Imagine John Lennon"]
    
cerca_canzoni(titoli)
load_song(current_song_index)
is_paused = False

🎶 Aggiunto: Bohemian Rhapsody (Remastered 2011) - Queen
🎶 Aggiunto: Around the World - Daft Punk
🎶 Aggiunto: Blinding Lights - The Weeknd
🎶 Aggiunto: Imagine (Remastered 2010) - John Lennon
▶️ Riproduzione: Bohemian Rhapsody (Remastered 2011) - Queen


In [5]:
# gesture recognition functions

# creazione di una zona di azione per la mano, meglio utilizzare 2 elementi: distanza tra polso e base medio e distanza tra polso e punta del pollice
# la zona di azione deve essere compresa tra i 50 e gli 80cm dalla telecamera

min_dist_medio = 0.15 
min_dist_pollice = 0.20
min_dist_nocche=0.1

max_dist_medio = 0.35
max_dist_pollice = 0.45
max_dist_nocche=1


def get_min_dist_nocche(hand_landmarks):
    
    polso_medio= math.dist([hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y ],[hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y ])
    polso_pollice= math.dist([hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y ],[hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y ])
    return polso_medio*(180/11)+polso_pollice*(-240/11)+0.1


def zona_attiva(hand_landmarks):
    #se il gesto avviene in questa zona allora si esegue l'azione
    
    #distanze
    polso_medio= math.dist([hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y ],[hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y ])
    polso_pollice= math.dist([hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y ],[hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y ])
    dist_nocche=math.dist([hand_landmarks.landmark[5].x, hand_landmarks.landmark[5].y ],[hand_landmarks.landmark[17].x, hand_landmarks.landmark[17].y ])
    
    if ((polso_medio > min_dist_medio and\
        polso_medio <max_dist_medio) or\
        (polso_pollice > min_dist_pollice and\
        polso_pollice <max_dist_pollice)) and\
            (dist_nocche>0.08):
            
    # and\
    #     dist_nocche>get_min_dist_nocche(hand_landmarks):
            return True


def check_open_hand(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    
    # IDs delle articolazioni delle dita (MediaPipe landmarks)
    finger_tips = [4, 8, 12, 16, 20]  # pollice, indice, medio, anulare, mignolo
    finger_pips = [2, 6, 10, 14, 18]   # articolazioni corrispondenti

    open_fingers = 0
    
    if hand_landmarks.landmark[0].y > min(all_y[1:]): #se il polso si trova più in basso rispetto a tutte le altre
        for i in range(5):
            tip = hand_landmarks.landmark[finger_tips[i]] #punta del dito corrente
            pip = hand_landmarks.landmark[finger_pips[i]] #articolazione del dito corrente
            
            # Per il pollice controlliamo la coordinata x
            if i == 0:  # pollice
                if (hand_landmarks.landmark[0].x < tip.x and tip.x > pip.x and hand_landmarks.landmark[1].x > hand_landmarks.landmark[0].x ) or \
                    (hand_landmarks.landmark[0].x > tip.x and tip.x < pip.x and hand_landmarks.landmark[1].x < hand_landmarks.landmark[0].x ):
                    open_fingers += 1
            # Per le altre dita controlliamo la coordinata y
            else:
                if tip.y < pip.y and\
                    abs(hand_landmarks.landmark[4].x-hand_landmarks.landmark[20].x)>0.1 and\
                    abs(hand_landmarks.landmark[0].y-hand_landmarks.landmark[12].y)>0.3    :  # la punta è sopra l'articolazione (dito aperto)
                    open_fingers += 1
        
        # Consideriamo la mano aperta se almeno 4 dita sono aperte (puoi cambiare a 5 per essere più preciso)
        return open_fingers >= 5
    else:
        return False


def check_thumbs_up(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_y = all_y[3:5] #joints 
    other_y = [all_y[0]]+all_y[5:] #all other joints 

    #con nocche laterali
    return max(thumb_y)<min(other_y) and not check_open_hand2(hand_landmarks)


def check_thumbs_down(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]

    thumb_y = all_y[2:5] #joints 
    other_y = [all_y[0]]+all_y[5:] #all other joints 
    
    return max(other_y)<min(thumb_y) and not check_open_hand2(hand_landmarks)



def check_thumbs_rx(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    indici_mid_nocche=[6, 10, 14, 18]

    all_but_nocche_thumb = [i for i in list(range(21)) if i not in indici_nocche+[2,3,4]]
    all_but_mid_nocche_thumb= [i for i in list(range(21)) if i not in indici_mid_nocche+[2,3,4]]
    
    # print(all_but_nocche_thumb, all_but_mid_nocche_thumb)   
    indici_tips=[8, 12, 16, 20]
    mid_nocche=[all_y[i] for i in indici_mid_nocche]
    
    all_under_nocche = all(
        min(all_y[i] for i in indici_nocche) <= all_y[i]
        for i in all_but_nocche_thumb
    )
    
    all_under_mid_nocche = all(
        min(all_y[i] for i in indici_mid_nocche) <= all_y[i]
        for i in all_but_mid_nocche_thumb
    )

    if min(thumb_x) > max(other_x) and\
        hand_landmarks.landmark[4].x > hand_landmarks.landmark[3].x and\
        (all_under_nocche or all_under_mid_nocche):
        return True
    
    return False


def check_thumbs_rx_old(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    nocche=[all_y[i] for i in indici_nocche]

    indici_tips=[8, 12, 16, 20]
    tips=[all_y[i] for i in indici_tips]

    dita_chiuse = any(
        all_y[tip] > all_y[nocca]
        for tip, nocca in zip(indici_tips, indici_nocche)
    )

    if min(thumb_x) > max(other_x) and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[17].y) < 0.2 and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[9].y) < 0.2 and\
            hand_landmarks.landmark[4].x > hand_landmarks.landmark[3].x and\
            dita_chiuse and\
            min(tips)>max(nocche):
        return True
    else:
        return False


def check_thumbs_sx(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    indici_mid_nocche=[6, 10, 14, 18]

    all_but_nocche_thumb = [i for i in list(range(21)) if i not in indici_nocche+[2,4]]
    all_but_mid_nocche_thumb= [i for i in list(range(21)) if i not in indici_mid_nocche+[2,4]]
    
    indici_tips=[8, 12, 16, 20]
    mid_nocche=[all_y[i] for i in indici_mid_nocche]
    
    all_under_nocche = all(
        min(all_y[i] for i in indici_nocche) <= all_y[i]
        for i in all_but_nocche_thumb
    )
    
    all_under_mid_nocche = all(
        min(all_y[i] for i in indici_mid_nocche) <= all_y[i]
        for i in all_but_mid_nocche_thumb
    )

    if max(thumb_x) < min(other_x) and\
        hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x and\
        (all_under_nocche or all_under_mid_nocche):
        return True
    
    return False


def check_thumbs_sx_old(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    indici_tips=[8, 12, 16, 20]
    
    nocche=[all_y[i] for i in indici_nocche]
    tips=[all_y[i] for i in indici_tips]
    
    dita_chiuse = any(
        all_y[tip] > all_y[nocca]
        for tip, nocca in zip(indici_tips, indici_nocche)
    )

    #inserire che il pollice deve stare a destra ripetto a tutti gli altri punti 4.x<all
    pollice_a_sx=all(
        hand_landmarks.landmark[4].x<=b
        for b in all_x
    )

    if  min(thumb_x) < max(other_x) and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[17].y) < 0.3 and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[9].y) < 0.5 and\
            hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x and\
            dita_chiuse and\
            pollice_a_sx and\
            min(tips)>max(nocche):
        return True
    else:
        return False


def check_open_hand2(hand_landmarks):
    #check se la mano è aperta per thumb
    # IDs delle articolazioni delle dita (MediaPipe landmarks)
    finger_tips = [4, 8, 12, 16, 20]  # pollice, indice, medio, anulare, mignolo
    finger_pips = [2, 6, 10, 14, 18]   # articolazioni corrispondenti

    open_fingers = 0
    
    for i in range(5):
        tip = hand_landmarks.landmark[finger_tips[i]] #punta del dito corrente
        pip = hand_landmarks.landmark[finger_pips[i]] #articolazione del dito corrente
        
        # Per il pollice controlliamo la coordinata x
        if i == 0:  # pollice
            if (hand_landmarks.landmark[0].x < tip.x and tip.x > pip.x and hand_landmarks.landmark[1].x > hand_landmarks.landmark[0].x ) or \
                (hand_landmarks.landmark[0].x > tip.x and tip.x < pip.x and hand_landmarks.landmark[1].x < hand_landmarks.landmark[0].x ):
                open_fingers += 0
        # Per le altre dita controlliamo la coordinata x
        else:
            if  abs(hand_landmarks.landmark[0].x-hand_landmarks.landmark[12].x)>0.2    :  # la punta è sopra l'articolazione (dito aperto)
                open_fingers += 1
    
    # Consideriamo la mano aperta se almeno 4 dita sono aperte (puoi cambiare a 5 per essere più preciso)
    return open_fingers >= 2




#### Codice per controllare la musica tramite gesture
- Per stoppare il codice premere `q` o, in caso non funzionasse, `restarta` il kernel

In [6]:
cooldown = 1 # Delay tra un gesto e l'altr (in secondi)
detection_delay = 2 # Delay tra gesto ed azione (in secondi)
movement_threshold = 0.05 # Soglia di movimento accettabile (valore da aggiustare)
cam=1 # Varibile della telecamera, 0 è la telecamera principale, 1 la seconda

In [ ]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import Image
import numpy as np
import pathlib
# Open_Palm, Victory, Thumb_Down, Thumb_Up, Closed_Fist



# Variabili globali
last_trigger_time = 0
detection_start_time = 0
is_detecting = False
current_gesture = None
last_landmarks_position = None # Per memorizzare la posizione precedente della mano



# Nuove variabili per tener traccia delle ripetizioni consecutive
consecutive_thumbs_up_count = 0
consecutive_thumbs_down_count = 0
last_gesture = None


# Percorsi dei file audio per i feedback sonori
path_start_sound = r"feedbacksounds\start_1.mp3"
path_start_1 = r"feedbacksounds\start_1.mp3"
path_start_i="feedbacksounds\\vol_{i}.mp3"
path_completed_sound = r"feedbacksounds\stop.mp3"

# Initialize pygame mixer
pygame.mixer.init()
# Load the sound file
DETECTION_START_SOUND  = pygame.mixer.Sound(path_start_sound)
GESTURE_COMPLETED_SOUND = pygame.mixer.Sound(path_completed_sound)
# Set volume (0.0 to 1.0)
DETECTION_START_SOUND.set_volume(0)
GESTURE_COMPLETED_SOUND.set_volume(1)

def handle_gesture(name, action_fn, hand_landmarks, frame):
    global last_trigger_time, detection_start_time, is_detecting, current_gesture
    global consecutive_thumbs_up_count, consecutive_thumbs_down_count, last_gesture
    global is_paused
    global last_landmarks_position # Aggiunto
    global movement_threshold # Aggiunto

    # --- Inizio del controllo di stabilità ---
    if is_detecting and current_gesture == name:
        # Calcola il centro della mano corrente (o qualsiasi punto di riferimento)
        current_center_x = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x
        current_center_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y

        if last_landmarks_position is not None:

            
            distance = math.dist([current_center_x, current_center_y], last_landmarks_position)
            
            if distance > movement_threshold:
                # Movimento eccessivo, resetta il rilevamento del gesto
                print(f"❌ Movimento eccessivo rilevato per {name}. Distanza: {distance:.4f}, Soglia: {movement_threshold:.4f}. Reset.")
                is_detecting = False
                current_gesture = None
                detection_start_time = 0 # Resetta il timer
                last_landmarks_position = None # Resetta la posizione iniziale
                # GESTURE_COMPLETED_SOUND.play() # Potresti riprodurre un suono di "errore"
                return # Esci dalla funzione, non processare il gesto
        
        # Se siamo all'inizio del rilevamento, salva la posizione iniziale
        if detection_start_time == time.time(): # Solo al primissimo frame di un nuovo rilevamento
             last_landmarks_position = [current_center_x, current_center_y]

    # --- Fine del controllo di stabilità ---

    if not is_detecting: # Se non stiamo rilevando O se è stato resettato per movimento
        is_detecting = True
        detection_start_time = time.time()
        current_gesture = name
        last_landmarks_position = [hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x,
                                   hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y] # Salva la posizione iniziale
        # Se stiamo già rilevando e il gesto è cambiato, resetta il timer
    # Questa parte è già presente e gestisce il cambio di gesto correttamente
    if current_gesture != name:
        is_detecting = True
        detection_start_time = time.time()
        current_gesture = name
        last_landmarks_position = [hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x,
                                   hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y] # Salva la nuova posizione iniziale
        if name != "Thumb_Up" and name != "Thumb_Up":
            DETECTION_START_SOUND.play()
        # return

    # Calcola il tempo trascorso dall'inizio del rilevamento
    elapsed_time = time.time() - detection_start_time

    # ... (il resto del codice della funzione handle_gesture rimane invariato) ...

    # Posizione per visualizzare il nome del gesto e il cerchio
    x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
    # Disegna il nome del gesto
    if not (name=="Open_Palm"):
        cv2.putText(frame, name.upper(), (400 , 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    elif is_paused:
        cv2.putText(frame, "PLAY", (400 , 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    elif not is_paused:
        cv2.putText(frame, "PAUSE", (400 , 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # Disegna il cerchio di caricamento
    radius = 30
    center = (x, y-20)
    # Disegna il cerchio di sfondo (grigio)
    cv2.circle(frame, center, radius, (100, 100, 100), 3)
    # Calcola l'angolo in base al tempo trascorso (da 0 a 360 gradi)
    angle = min(elapsed_time / detection_delay * 360, 360)
    # Disegna l'arco di caricamento (verde)
    start_angle = -90  # Inizia dall'alto
    end_angle = start_angle + angle
    # Disegna l'arco di caricamento punto per punto
    for i in range(int(start_angle), int(end_angle)):
        rad = math.radians(i)
        x1 = int(center[0] + radius * math.cos(rad))
        y1 = int(center[1] + radius * math.sin(rad))
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), 3)

    # Se il tempo di attesa è completato, esegui l'azione
    if elapsed_time >= detection_delay:
        action_fn()
        # print(f"{name.upper()} RILEVATO!")
        # GESTURE_COMPLETED_SOUND.play()

        # Aggiorna i contatori per i gesti consecutivi
        if name == "Thumb_Up":
            if last_gesture == "Thumb_Up":
                consecutive_thumbs_up_count += 1
                if volume==1:
                    pygame.mixer.Sound(path_start_i.format(i=0)).play()
                else:
                    pygame.mixer.Sound(path_start_i.format(i = int(volume * 10))).play()
            else:
                consecutive_thumbs_up_count = 1
                pygame.mixer.Sound(path_start_i.format(i = int(volume * 10))).play()
            consecutive_thumbs_down_count = 0

        elif name == "Thumb_Up":
            if last_gesture == "Thumb_Up":
                consecutive_thumbs_down_count += 1
                if volume==0.0:
                    pygame.mixer.Sound(path_start_i.format(i=10)).play()
                else:
                    pygame.mixer.Sound(path_start_i.format(i = int(volume * 10))).play()
            else:
                consecutive_thumbs_down_count = 1
                pygame.mixer.Sound(path_start_i.format(i = int(volume * 10))).play()
            consecutive_thumbs_up_count = 0
        else:
            consecutive_thumbs_up_count = 0
            consecutive_thumbs_down_count = 0
            pygame.mixer.Sound(path_completed_sound).play()

        last_gesture = name
        last_trigger_time = time.time()
        is_detecting = False
        current_gesture = None
        last_landmarks_position = None # Resetta la posizione iniziale anche qui dopo l'esecuzione del gesto

def estimate_hand_side(landmarks, handedness):
    WRIST = 0
    INDEX_MCP = 5
    PINKY_MCP = 17

    wrist = np.array([landmarks[WRIST].x, landmarks[WRIST].y, landmarks[WRIST].z])
    index_mcp = np.array([landmarks[INDEX_MCP].x, landmarks[INDEX_MCP].y, landmarks[INDEX_MCP].z])
    pinky_mcp = np.array([landmarks[PINKY_MCP].x, landmarks[PINKY_MCP].y, landmarks[PINKY_MCP].z])

    vec1 = index_mcp - wrist
    vec2 = pinky_mcp - wrist

    normal = np.cross(vec1, vec2)

    # Se la mano è sinistra, invertiamo la normale
    if handedness == 'Left':
        normal = -normal

    if normal[2] > 0:
        return "PALM"
    else:
        return "BACK"




# Caricamento modello Gesture Recognizer da buffer
model_path = Path(r"gesture_recognizer.task").resolve()
base_options = python.BaseOptions(model_asset_buffer=open(model_path, "rb").read())
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

# Inizializzazione di MediaPipe e della webcam
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.7)








# --- Caricamento modello Gesture Recognizer ---
model_path = pathlib.Path(r"gesture_recognizer.task").resolve()
base_options = python.BaseOptions(model_asset_buffer=open(model_path, "rb").read())
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

# Inizializzazione di MediaPipe e della webcam
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.7)

# --- Apertura webcam ---
cap = cv2.VideoCapture(cam)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if is_detecting and (time.time() - detection_start_time > detection_delay * 1.5):
        is_detecting = False
        current_gesture = None
    
    # Converti l'immagine in RGB (MediaPipe lavora in RGB)
    frame = cv2.flip(frame, 1)  # Capovolgi l'immagine per una visualizzazione speculare
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks and results.multi_handedness:
        
        for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            
            #check distanze
            dist_nocche= math.dist([hand_landmarks.landmark[5].x, hand_landmarks.landmark[5].y ],[hand_landmarks.landmark[17].x, hand_landmarks.landmark[17].y ])

            if zona_attiva(hand_landmarks):
                landmark_color = (0, 255, 0)  # Verde
                connection_color = (100, 100, 100)  
            if not zona_attiva(hand_landmarks):
                landmark_color = (0, 0, 255)  # Rosso
                connection_color = (0, 0, 255)
            mp_draw.draw_landmarks(
                frame, 
                hand_landmarks, 
                mp_hands.HAND_CONNECTIONS,
                mp_draw.DrawingSpec(color=landmark_color, thickness=2, circle_radius=2),
                mp_draw.DrawingSpec(color=connection_color, thickness=5, circle_radius=2))
            
            # Controlla se possiamo saltare il cooldown per i gesti "thumbs_up" e "thumbs_down"
            skip_cooldown = False
            
        
            # Riconoscimento gesture
            result = recognizer.recognize(mp_image)

            # Disegna il nome del gesto sul frame
            if result.gestures:
                gesture = result.gestures[0][0]  # Primo gesto con punteggio più alto
                gesture_name = gesture.category_name
                score = gesture.score
                if last_gesture == "Thumb_Up" and consecutive_thumbs_up_count >= 2:
                    if check_thumbs_up(hand_landmarks):
                        skip_cooldown = True
            
                if last_gesture == "Thumb_Up" and consecutive_thumbs_down_count >= 2:
                    if check_thumbs_down(hand_landmarks):
                        skip_cooldown = True

                # Verifica il cooldown o se possiamo saltarlo
                if zona_attiva(hand_landmarks) and ((time.time() - last_trigger_time) > cooldown or skip_cooldown) :
                    if gesture_name=="Open_Palm" and estimate_hand_side(hand_landmarks.landmark, hand_handedness.classification[0].label) == "PALM" and ((estimate_hand_side(hand_landmarks.landmark, hand_handedness.classification[0].label) == "BACK")==False):
                        handle_gesture("Open_Palm", play_pause, hand_landmarks, frame)
                    elif gesture_name=="Thumb_Up":
                        handle_gesture("Thumb_Up", volume_up, hand_landmarks, frame)
                    elif gesture_name== "Closed_First":
                        handle_gesture("Closed_First", previous_song, hand_landmarks, frame)
                    elif gesture_name== "Victory":
                        handle_gesture("Victory", next_song, hand_landmarks, frame)
                    elif gesture_name=="Thumb_Down":
                        handle_gesture("Thumb_Down", volume_down, hand_landmarks, frame)
                
                cv2.putText(frame, f"{gesture_name} ({score:.2f})", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                

            cv2.imshow("Handle music with gestures + palm ", frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                pygame.mixer.music.pause()
                cap.release()
                cv2.destroyAllWindows()
                break



